# The Cancer Gene Atlas Cancer (TCGA) - Head and Neck Squamous Cell Carcinoma (HNSC)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Thu Apr 26 16:39:46 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Head and Neck squamous cell carcinoma [HNSC]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-BA-6871,200,HNSC,TCGA,bd62a587-6fe1-47ce-918d-055f2b38bb0e,BA,75.0,Base of tongue,145,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,"Yes, History of Prior Malignancy","Kidney, Prostate",Kidney Clear Cell Renal Carcinoma,"Prior Malignancy, Prior Malignancy",2005.0,NaN,1950.0
TCGA-BB-7871,201,HNSC,TCGA,9dd8b7cc-d094-474b-9573-cbff4ab233c8,BB,64.0,Base of tongue,215,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,1961.0
TCGA-BB-4225,202,HNSC,TCGA,c1f50a22-38df-41cc-a1f4-f7985504a7ac,BB,73.0,Base of tongue,54,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,2001.0,NaN,1948.0
TCGA-BB-4228,203,HNSC,TCGA,cac30b32-03ef-4ecb-88d8-d5577ab6b2a6,BB,50.0,Base of tongue,54,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,1969.0
TCGA-BB-7861,204,HNSC,TCGA,1ae087ac-63f0-4cf9-b698-f5c13e1f4681,BB,56.0,Base of tongue,215,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(528, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_HNSC_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,bb9fff02-9a50-4b97-90d8-daecfa9104a0,0eeec485-9070-4099-b6a0-be5158323751,f254869c-d6e3-45b7-8144-20be5030a3f1,f3dca814-de43-4984-ba7a-0641ac206f93,22acd051-cc4f-4aa3-a91b-83787ebbdd78,b007ae52-24d8-48ca-bd6b-1d1c1d3f22e1,0d0ead46-415e-4c16-8901-ed5c202d942f,bb672329-d237-4add-be66-e14f73178fe3,e51c0a53-57b3-49e8-8122-7f37a633fed0,d5294af1-ae65-4946-b290-df8c452664b8,...,f5bec5f9-2344-4fac-b935-89ae58f1c517,d82211c8-7aee-4e19-bb00-d7bf42a549c8,ab715bb7-06cd-44ed-85b4-aafccf6111ff,873be7d3-e993-4f55-9cd9-f135b3e07b0a,11c15974-2a2b-4680-b52c-5bf81e17f3ee,85cb0fd1-5602-4681-bff4-0f9719aa3715,1147624d-cf30-477e-99f9-f595e8d6f864,5c39715d-2463-42cc-9685-3cdd4b547548,ef9758e6-6a55-42d7-b0ee-74163dd5b3d0,184149e5-4187-403e-95fa-87f7b87f2911
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,146.3370,166.8510,262.153000,146.62900,186.50100,195.870000,312.55100,161.01200,148.70300,129.3780,...,134.139000,205.88500,228.710000,283.717000,227.6610,162.88900,102.11900,298.73700,216.466000,98.34630
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.8108,51.6051,25.545300,38.33810,64.49400,48.399200,60.18700,43.43650,18.36230,40.9492,...,1.185710,55.29230,0.687770,8.256860,0.0000,26.82920,0.00000,42.82280,61.190300,81.00350
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,24.1135,38.9485,21.006900,18.53100,23.00730,15.587900,15.92870,18.66980,19.80530,15.8716,...,22.128600,25.44980,12.832300,23.064200,33.0766,32.43450,23.72990,17.93020,33.407200,18.99440
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,50.6584,56.4056,48.256100,54.52000,73.83610,126.148000,163.28000,43.83990,32.22120,48.6841,...,89.240400,70.95080,60.538100,75.762300,68.0312,133.92900,146.00000,32.06310,50.851300,48.50050
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,3.5047,10.8825,0.222934,4.60784,1.10273,0.361295,8.68803,4.48847,5.15447,1.3795,...,0.000383,2.88628,0.743421,0.136801,0.0000,3.09853,1.71307,3.76532,0.181912,1.18989


In [19]:
matrix.shape

(199169, 566)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,bb9fff02-9a50-4b97-90d8-daecfa9104a0,0eeec485-9070-4099-b6a0-be5158323751,f254869c-d6e3-45b7-8144-20be5030a3f1,f3dca814-de43-4984-ba7a-0641ac206f93,22acd051-cc4f-4aa3-a91b-83787ebbdd78,b007ae52-24d8-48ca-bd6b-1d1c1d3f22e1,0d0ead46-415e-4c16-8901-ed5c202d942f,bb672329-d237-4add-be66-e14f73178fe3,e51c0a53-57b3-49e8-8122-7f37a633fed0,d5294af1-ae65-4946-b290-df8c452664b8,...,f5bec5f9-2344-4fac-b935-89ae58f1c517,d82211c8-7aee-4e19-bb00-d7bf42a549c8,ab715bb7-06cd-44ed-85b4-aafccf6111ff,873be7d3-e993-4f55-9cd9-f135b3e07b0a,11c15974-2a2b-4680-b52c-5bf81e17f3ee,85cb0fd1-5602-4681-bff4-0f9719aa3715,1147624d-cf30-477e-99f9-f595e8d6f864,5c39715d-2463-42cc-9685-3cdd4b547548,ef9758e6-6a55-42d7-b0ee-74163dd5b3d0,184149e5-4187-403e-95fa-87f7b87f2911
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,146.3370,166.8510,262.153000,146.62900,186.50100,195.870000,312.55100,161.01200,148.70300,129.3780,...,134.139000,205.88500,228.710000,283.717000,227.6610,162.88900,102.11900,298.73700,216.466000,98.34630
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.8108,51.6051,25.545300,38.33810,64.49400,48.399200,60.18700,43.43650,18.36230,40.9492,...,1.185710,55.29230,0.687770,8.256860,0.0000,26.82920,0.00000,42.82280,61.190300,81.00350
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,24.1135,38.9485,21.006900,18.53100,23.00730,15.587900,15.92870,18.66980,19.80530,15.8716,...,22.128600,25.44980,12.832300,23.064200,33.0766,32.43450,23.72990,17.93020,33.407200,18.99440
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,50.6584,56.4056,48.256100,54.52000,73.83610,126.148000,163.28000,43.83990,32.22120,48.6841,...,89.240400,70.95080,60.538100,75.762300,68.0312,133.92900,146.00000,32.06310,50.851300,48.50050
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,3.5047,10.8825,0.222934,4.60784,1.10273,0.361295,8.68803,4.48847,5.15447,1.3795,...,0.000383,2.88628,0.743421,0.136801,0.0000,3.09853,1.71307,3.76532,0.181912,1.18989


In [22]:
matrix.shape

(79930, 566)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,146.3370,166.8510,262.153000,146.62900,186.50100,195.870000,312.55100,161.01200,148.70300,129.3780,...,134.139000,205.88500,228.710000,283.717000,227.6610,162.88900,102.11900,298.73700,216.466000,98.34630
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.8108,51.6051,25.545300,38.33810,64.49400,48.399200,60.18700,43.43650,18.36230,40.9492,...,1.185710,55.29230,0.687770,8.256860,0.0000,26.82920,0.00000,42.82280,61.190300,81.00350
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,24.1135,38.9485,21.006900,18.53100,23.00730,15.587900,15.92870,18.66980,19.80530,15.8716,...,22.128600,25.44980,12.832300,23.064200,33.0766,32.43450,23.72990,17.93020,33.407200,18.99440
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,50.6584,56.4056,48.256100,54.52000,73.83610,126.148000,163.28000,43.83990,32.22120,48.6841,...,89.240400,70.95080,60.538100,75.762300,68.0312,133.92900,146.00000,32.06310,50.851300,48.50050
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,3.5047,10.8825,0.222934,4.60784,1.10273,0.361295,8.68803,4.48847,5.15447,1.3795,...,0.000383,2.88628,0.743421,0.136801,0.0000,3.09853,1.71307,3.76532,0.181912,1.18989


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
ARF5,146.3370,166.8510,262.153000,146.62900,186.50100,195.870000,312.55100,161.01200,148.70300,129.3780,...,134.139000,205.88500,228.710000,283.717000,227.6610,162.88900,102.11900,298.73700,216.466000,98.34630
M6PR,35.8108,51.6051,25.545300,38.33810,64.49400,48.399200,60.18700,43.43650,18.36230,40.9492,...,1.185710,55.29230,0.687770,8.256860,0.0000,26.82920,0.00000,42.82280,61.190300,81.00350
ESRRA,24.1135,38.9485,21.006900,18.53100,23.00730,15.587900,15.92870,18.66980,19.80530,15.8716,...,22.128600,25.44980,12.832300,23.064200,33.0766,32.43450,23.72990,17.93020,33.407200,18.99440
FKBP4,50.6584,56.4056,48.256100,54.52000,73.83610,126.148000,163.28000,43.83990,32.22120,48.6841,...,89.240400,70.95080,60.538100,75.762300,68.0312,133.92900,146.00000,32.06310,50.851300,48.50050
CYP26B1,3.5047,10.8825,0.222934,4.60784,1.10273,0.361295,8.68803,4.48847,5.15447,1.3795,...,0.000383,2.88628,0.743421,0.136801,0.0000,3.09853,1.71307,3.76532,0.181912,1.18989


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_hnsc_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
A1BG,0.062251,0.062251,0.062251,0.062251,0.044661,0.062251,0.062251,0.062251,0.062251,0.011253,...,0.062251,0.062251,0.023040,0.062251,0.062251,0.062251,0.062251,0.009858,0.062251,0.062251
A1CF,0.012577,0.011653,0.005002,0.011620,0.011620,0.008083,0.004946,0.009001,0.011620,0.011620,...,0.011620,0.019086,0.011620,0.008873,0.011620,0.011620,0.019218,0.005483,0.007657,0.021426
A2M,49.598700,227.726000,6.025550,41.644000,64.476100,39.980500,56.375300,27.326500,12.856900,37.640800,...,16.556500,67.153000,30.093400,4.732030,41.169600,74.023000,14.898100,27.623800,35.969800,94.779000
A2ML1,102.648000,81.016700,48.131300,95.202000,11.833000,19.972000,96.703600,67.351200,153.066000,27.931600,...,16.914600,3.486840,39.208400,21.153800,10.825700,62.761700,36.076700,1.986870,114.549000,11.792700
A3GALT2,0.015955,0.065469,0.065469,0.065469,0.019707,0.017025,0.065469,0.065469,0.065469,0.016483,...,0.065469,0.065469,0.065469,0.065469,0.065469,0.019072,0.065469,0.065469,0.065469,0.055151


In [34]:
normalized_matrix.shape

(19165, 566)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [36]:
samples = normalized_matrix.columns.tolist()

In [37]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [38]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [39]:
normalized_matrix.columns = samples

In [40]:
normalized_matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
A1BG,-1.066010,-1.052693,-1.041670,-1.074151,-1.185855,-1.011542,-1.021843,-1.066177,-1.017578,-1.790202,...,-0.957526,-1.040130,-1.402194,-1.034467,-0.963507,-1.029343,-0.949232,-1.836561,-1.048119,-0.914864
A1CF,-1.760597,-1.780405,-2.136664,-1.803097,-1.770588,-1.898114,-2.121766,-1.906044,-1.746524,-1.776266,...,-1.686472,-1.553565,-1.699476,-1.880522,-1.692452,-1.758289,-1.459658,-2.091348,-1.958184,-1.378059
A2M,1.835314,2.510573,0.944180,1.751255,1.973610,1.796160,1.935100,1.576261,1.297412,1.734193,...,1.467296,1.992790,1.713795,0.846434,1.856924,2.045877,1.429753,1.610949,1.713673,2.267702
A2ML1,2.151194,2.061735,1.846611,2.110349,1.237307,1.494733,2.169453,1.968022,2.373155,1.604630,...,1.476589,0.708156,1.828703,1.496775,1.276803,1.974205,1.813849,0.467835,2.216726,1.362603
A3GALT2,-1.657257,-1.030802,-1.019779,-1.052261,-1.541170,-1.574604,-0.999952,-1.044286,-0.995687,-1.624419,...,-0.935636,-1.018239,-0.948639,-1.012576,-0.941616,-1.543100,-0.927341,-1.014298,-1.026228,-0.967459


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [41]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [42]:
normalized_matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
A1BG,-0.949606,-1.010278,-0.943752,-0.913600,-1.132669,-1.012545,-1.074617,-0.931430,-0.854015,-1.663956,...,-1.219494,-1.045352,-1.738001,-0.821009,-1.255033,-1.034701,-1.208848,-1.771917,-1.044826,-1.041578
A1CF,-1.624521,-1.705102,-2.041290,-1.643848,-1.727915,-1.858540,-2.176311,-1.827640,-1.593882,-1.650631,...,-2.104645,-1.561828,-2.036180,-1.670357,-2.043870,-1.773748,-1.840226,-2.023364,-1.967004,-1.482405
A2M,1.881434,2.482464,1.019332,1.837912,1.937193,1.750801,1.881124,1.611979,1.474928,1.691464,...,1.380288,1.942303,1.629943,1.047253,1.724927,2.028805,1.359562,1.547423,1.653814,2.182017
A2ML1,2.186796,2.035385,1.898842,2.184390,1.215741,1.455601,2.143435,2.002599,2.489135,1.569208,...,1.390567,0.628804,1.755798,1.612744,1.153752,1.955952,1.759316,0.472661,2.202599,1.166504
A3GALT2,-1.522196,-0.993168,-0.923636,-0.891327,-1.498858,-1.554565,-1.057641,-0.909982,-0.832213,-1.520081,...,-1.194680,-1.022013,-1.204042,-0.792152,-1.230705,-1.539257,-1.185737,-0.971188,-1.024285,-1.083263


# Normalize Matrix (z-score the rows)

In [43]:
uf.zscore(normalized_matrix, 'row')

In [44]:
normalized_matrix.head()

,TCGA-CQ-6229-01,TCGA-CQ-6227-01,TCGA-CQ-6225-01,TCGA-CQ-6218-01,TCGA-CQ-6219-01,TCGA-CQ-6228-01,TCGA-CV-6941-01,TCGA-CV-6952-01,TCGA-CV-6953-01,TCGA-CV-6954-01,...,TCGA-CV-7409-01,TCGA-CQ-7067-01,TCGA-DQ-7589-01,TCGA-BB-7870-01,TCGA-BB-7871-01,TCGA-BB-7862-01,TCGA-BB-7864-01,TCGA-BB-7872-01,TCGA-DQ-7595-01,TCGA-BB-7861-01
A1BG,0.664353,0.067221,0.721970,1.018726,-1.137345,0.044911,-0.565997,0.843244,1.605152,-6.366241,...,-1.991873,-0.277976,-7.094989,1.930000,-2.341640,-0.173143,-1.887089,-7.428785,-0.272792,-0.240834
A1CF,0.659399,0.396270,-0.701513,0.596289,0.321779,-0.104764,-1.142410,-0.003865,0.759450,0.574139,...,-0.908393,0.864117,-0.684828,0.509727,-0.709938,0.172116,-0.044961,-0.642978,-0.458942,1.123464
A2M,0.282848,1.877762,-2.004857,0.167355,0.430811,-0.063805,0.282023,-0.432189,-0.795871,-0.221264,...,-1.047012,0.444372,-0.384519,-1.930767,-0.132467,0.673915,-1.102010,-0.603498,-0.321175,1.080484
A2ML1,0.763613,0.545136,0.348110,0.760142,-0.637570,-0.291464,0.701046,0.497827,1.199873,-0.127535,...,-0.385304,-1.484489,0.141705,-0.064714,-0.727016,0.430517,0.146782,-1.709795,0.786416,-0.708616
A3GALT2,-2.347104,0.200641,0.535499,0.691096,-2.234713,-2.502993,-0.109854,0.601258,0.975786,-2.336920,...,-0.769820,0.061725,-0.814906,1.168716,-0.943311,-2.429271,-0.726751,0.306496,0.050786,-0.233250


# Merge Like Column (by taking the mean)

In [45]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [46]:
normalized_matrix.shape

(19165, 566)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [47]:
uf.mapgenesymbols(normalized_matrix)

In [48]:
normalized_matrix.shape

(18737, 566)

# Merge Duplicate Genes By Rows

In [49]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [50]:
normalized_matrix.shape

(18648, 566)

# Histogram of First Sample

In [51]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [52]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [53]:
filename = path+'tcga_hnsc_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [54]:
gene_list = uf.createGeneList(normalized_matrix)

In [55]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [56]:
gene_list.shape

(18648, 2)

# Save Gene List

In [57]:
filename = path+'tcga_hnsc_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [58]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [59]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [60]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-4P-AA8J-01,66,MALE,MX,N2c,T2,Stage IVA,Head & Neck Squamous Cell Carcinoma,Primary Solid Tumor,Head and Neck,G2
TCGA-BA-4074-01,69,MALE,M0,N2c,T2,Stage IVA,Head & Neck Squamous Cell Carcinoma,Primary Solid Tumor,Head and Neck,G3
TCGA-BA-4075-01,49,MALE,M0,N0,T3,Stage III,Head & Neck Squamous Cell Carcinoma,Primary Solid Tumor,Head and Neck,G2
TCGA-BA-4076-01,39,MALE,NaN,NX,TX,NaN,Head & Neck Squamous Cell Carcinoma,Primary Solid Tumor,Head and Neck,G2
TCGA-BA-4077-01,45,FEMALE,M0,N0,T4a,Stage IVA,Head & Neck Squamous Cell Carcinoma,Primary Solid Tumor,Head and Neck,G2


In [61]:
attribute_list.shape

(566, 10)

# Save Attribute List

In [62]:
filename = path+'tcga_hnsc_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [63]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [64]:
standard_matrix.head()

,TCGA-4P-AA8J-01,TCGA-BA-4074-01,TCGA-BA-4075-01,TCGA-BA-4076-01,TCGA-BA-4077-01,TCGA-BA-4078-01,TCGA-BA-5149-01,TCGA-BA-5151-01,TCGA-BA-5152-01,TCGA-BA-5153-01,...,TCGA-UF-A7JJ-01,TCGA-UF-A7JK-01,TCGA-UF-A7JO-01,TCGA-UF-A7JS-01,TCGA-UF-A7JT-01,TCGA-UF-A7JV-01,TCGA-UP-A6WW-01,TCGA-WA-A7GZ-01,TCGA-WA-A7GZ-11,TCGA-WA-A7H4-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.999602,-0.939713,-0.879676,0.413276,-0.434647,-0.706595,0.667795,0.734982,0.848174,-0.597115,...,-0.561815,0.310774,0.399127,0.812793,0.830498,0.621836,0.176518,-0.625376,-0.335753,-0.812599
A1CF,-0.177066,0.947035,0.823141,-0.805787,-0.886994,0.975350,0.098496,0.847917,0.186826,-0.159407,...,0.462528,-0.777548,0.709981,0.854980,-0.117006,0.413012,-0.307768,0.229247,0.932853,0.176262
A2M,0.496281,-0.821459,-0.920358,-0.881500,-0.556543,0.340751,-0.206900,0.623549,-0.093849,0.354894,...,0.376128,-0.814395,0.015660,-0.086780,-0.040891,-0.422323,-0.291644,-0.605999,0.591747,0.711962
A2ML1,-0.280218,-0.806484,-0.923025,0.200261,-0.248417,-0.548609,-0.788832,0.642108,-0.032946,0.012996,...,-0.082400,-0.117717,-0.135392,-0.693439,-0.470946,-0.407358,0.246227,0.076574,0.932156,-0.357924
A3GALT2,-0.119960,0.696425,0.703501,0.466645,-0.699201,0.742390,-0.731013,0.625697,0.707047,0.484318,...,-0.162346,0.374745,-0.692151,0.699956,-0.663906,-0.688638,0.745924,-0.130539,-0.349554,0.869723


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [65]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [66]:
filename = path+'tcga_hnsc_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [67]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [68]:
tertiary_matrix.head()

,TCGA-4P-AA8J-01,TCGA-BA-4074-01,TCGA-BA-4075-01,TCGA-BA-4076-01,TCGA-BA-4077-01,TCGA-BA-4078-01,TCGA-BA-5149-01,TCGA-BA-5151-01,TCGA-BA-5152-01,TCGA-BA-5153-01,...,TCGA-UF-A7JJ-01,TCGA-UF-A7JK-01,TCGA-UF-A7JO-01,TCGA-UF-A7JS-01,TCGA-UF-A7JT-01,TCGA-UF-A7JV-01,TCGA-UP-A6WW-01,TCGA-WA-A7GZ-01,TCGA-WA-A7GZ-11,TCGA-WA-A7H4-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [69]:
filename = path+'tcga_hnsc_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [70]:
name = 'tcga_hnsc_gene_up_set'

In [71]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [72]:
name = 'tcga_hnsc_gene_down_set'

In [73]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [74]:
name = 'tcga_hnsc_attribute_up_set'

In [75]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [76]:
name = 'tcga_hnsc_attribute_down_set'

In [77]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [78]:
attribute_normalized_matrix = normalized_matrix.copy()

In [79]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [80]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [81]:
attribute_similarity_matix.head()

,"(Donor:TCGA-4P-AA8J, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-4074, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG3)","(Donor:TCGA-BA-4075, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-4076, Age:39.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-4077, Age:45.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-4078, Age:83.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-5149, Age:47.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-5151, Age:72.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG1)","(Donor:TCGA-BA-5152, Age:56.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-BA-5153, Age:51.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)",...,"(Donor:TCGA-UF-A7JJ, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG1)","(Donor:TCGA-UF-A7JK, Age:59.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-UF-A7JO, Age:79.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-UF-A7JS, Age:59.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-UF-A7JT, Age:72.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG3)","(Donor:TCGA-UF-A7JV, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-UP-A6WW, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-WA-A7GZ, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-WA-A7GZ, Age:58.0, Gender:MALE, SampleType:Solid Tissue Normal, Stage:nan, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)","(Donor:TCGA-WA-A7H4, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG3)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-4P-AA8J, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Head & Neck Squamous Cell Carcinoma, Tissue:Head and Neck, Tumor:nan, GradeG2)",1.000000,0.055671,0.030442,-0.025119,0.070385,0.046858,-0.070249,-0.183927,0.057351,-0.025911,...,0.213238,0.023733,0.104057,0.243724,0.16

# Save Attribute Similarity Matrix

In [82]:
filename = path+'tcga_hnsc_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [83]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [84]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [85]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.050570,-0.134690,0.204343,0.060099,0.010328,-0.014040,-0.025933,0.025215,0.059358,...,0.059266,0.061523,-0.000516,-0.003265,0.065171,-0.001167,-0.028088,-0.072386,0.030568,0.010968
A1CF,0.050570,1.000000,-0.097517,-0.032832,0.034835,-0.026830,-0.003143,0.080904,-0.086688,-0.056844,...,0.163952,0.168027,0.026293,0.030066,0.026886,0.112097,-0.034891,-0.103234,-0.063039,0.011782
A2M,-0.134690,-0.097517,1.000000,-0.158128,-0.137030,0.003566,-0.129113,0.019082,-0.026044,-0.150400,...,-0.253439,-0.346669,0.160568,-0.016991,-0.153506,-0.173143,0.280072,0.287795,0.269857,0.106641
A2ML1,0.204343,-0.032832,-0.158128,1.000000,0.086531,0.066830,0.063305,-0.178958,0.333352,0.195733,...,0.028756,-0.001326,-0.028526,-0.003050,0.053855,0.092460,0.141870,-0.162339,0.202238,0.185004
A3GALT2,0.060099,0.034835,-0.137030,0.086531,1.000000,-0.017122,0.036786,0.014933,0.150178,0.017780,...,0.086943,0.124631,0.016575,-0.008508,0.030254,0.157522,0.068550,-0.107517,0.018177,0.060076


# Save Gene Similarity Matrix

In [86]:
filename = path+'tcga_hnsc_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [87]:
name = 'tcga_hnsc_gene_attribute_edge_list'

In [88]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  566 Out of 566   

 The number of statisticaly relevent gene-attribute associations is: 527943
